In [1]:
%pylab notebook
import matplotlib.pyplot as plt
import numpy as np
import sys, os
import pmoired

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
oi = pmoired.OI('../oifits_data/MYSTIC_L2.2023Jul21.RW_Cep.MIRCX_IDL.all_lbd.AVG10m.oifits')

loadOI: loading MYSTIC_L2.2023Jul21.RW_Cep.MIRCX_IDL.all_lbd.AVG10m.oifits
  > insname: "MIRCX" targname: "RW Cep" pipeline: ""
  > MJD: (208,) [ 60146.43827034789 .. 60146.47376080186 ]
  > E1-E2-S1-S2-W1-W2 | WL: (56,) [ 1.983 .. 2.397 ] um (R~290) {'T3': 20, 'VIS': 15, 'VIS2': 15} | TELL: False 


In [ ]:
oi.setupFit({'obs':['V2', 'T3PHI'], 
            'max error':{'T3PHI':10},
            'max relative error':{'V2':0.25},
            'wl ranges': [(1.9, 2.29111111)]})

m = {'star,alpha':       1.5, 
'star,diam':   2.7, 
'star,incl':   20, 
'star,projang':0, 
'star,profile':'$MU**$star,alpha',
}
prior = []
expl = {'rand':{'star,projang':(-90, 90)}}
# add spots
Nspots = 3
for i in range(Nspots):
    k = 'spot%02d'%(i+1)
    m.update({k+',fwhm':m['star,diam']/8, k+',f':0.0, 
              k+',x':'$'+k+',dist*np.sin($'+k+',PA)', 
              k+',y':'$'+k+',dist*np.cos($'+k+',PA)', 
              k+',dist':m['star,diam']/2*np.random.rand(), 
              k+',PA':2*np.pi*np.random.rand(), 
              })
    prior += [(k+',dist', '<', 'star,diam/2'), 
              (k+',dist', '>', 0), 
              (k+',fwhm', '>', 'star,diam/10')]
    expl['rand'].update({k+',dist':(m['star,diam']/8, m['star,diam']/2),
                         k+',PA':(2*np.pi*i/Nspots, 2*np.pi*(i+1)/Nspots),
                        })
oi.gridFit(model=m, expl=expl, Nfits=24, prior=prior)
oi.show(logV=True, imFov=3, imMax='99', imLegend=False, spectro=False)

Wed Aug 14 23:31:30 2024: running 24 fits on 12 processes


In [ ]:
mk = {'spot01,PA':   1.542, # +/- 0.015
'spot01,dist': 0.892, # +/- 0.019
'spot01,f':    0.111, # +/- 0.013
'spot01,fwhm': 0.923, # +/- 0.030
'spot02,PA':   2.976, # +/- 0.020
'spot02,dist': 0.700, # +/- 0.023
'spot02,f':    0.105, # +/- 0.011
'spot02,fwhm': 0.796, # +/- 0.044
'spot03,PA':   4.899, # +/- 0.014
'spot03,dist': 0.629, # +/- 0.030
'spot03,f':    0.076, # +/- 0.012
'spot03,fwhm': 0.654, # +/- 0.068
'star,alpha':  0.595, # +/- 0.085
'star,diam':   2.518, # +/- 0.022
'star,incl':   21.08, # +/- 1.43
'star,projang':-165.05, # +/- 3.38
'spot01,x':    '$spot01,dist*np.sin($spot01,PA)',
'spot01,y':    '$spot01,dist*np.cos($spot01,PA)',
'spot02,x':    '$spot02,dist*np.sin($spot02,PA)',
'spot02,y':    '$spot02,dist*np.cos($spot02,PA)',
'spot03,x':    '$spot03,dist*np.sin($spot03,PA)',
'spot03,y':    '$spot03,dist*np.cos($spot03,PA)',
'star,profile':'$MU**$star,alpha',
}


oi.setupFit({'obs':['V2', 'T3PHI'], 
            'max error':{'T3PHI':10},
            'max relative error':{'V2':0.25},
            })

doNotFit = list(mk.keys())

if True:   
    # https://www.gemini.edu/observing/resources/near-ir-resources/spectroscopy/co-lines-and-band-heads
    # 12C16O
    CO = [2.2935, 2.3227, 2.3525]
    mk.update({ 'envVEL':              1900,
                'env,f':              0.,
                'env,line_1_f':       1.0,
                'env,line_1_truncexp': 10.0,
                'env,line_1_wl0':     str(CO[0])+'*(1+$envVEL/3.e5)',
                'env,line_2_f':       1.0,
                'env,line_2_truncexp':'$env,line_1_truncexp',
                'env,line_2_wl0':     str(CO[1])+'*(1+$envVEL/3.e5)',
                'env,line_3_f':       1.0,
                'env,line_3_truncexp':'$env,line_1_truncexp',
                'env,line_3_wl0':     str(CO[2])+'*(1+$envVEL/3.e5)',
                'env,diamin':'$star,diam',
                'env,diamout':'2*$star,diam',
               'env,profile':'$R**-2',
               'env,az amp1':0.3,
               'env,az projang1':45,    
               'env,az amp2':0.3,
               'env,az projang2':45,    
               'env,az amp3':0.3,
               'env,az projang3':45,    
            })
    # -- define global absorption
    mk.update({'ABS1':0.8, 'ABS2':'$ABS1', 'ABS3':'$ABS1'})
    for c in ['star', 'spot01', 'spot02', 'spot03']:
        if c=='star':
            mk[c+',line_1_f'] = '-$ABS1'
            mk[c+',line_2_f'] = '-$ABS2'
            mk[c+',line_3_f'] = '-$ABS3'
        else:
            mk[c+',line_1_f'] = '-$ABS1*$'+c+',f'
            mk[c+',line_2_f'] = '-$ABS2*$'+c+',f'
            mk[c+',line_3_f'] = '-$ABS3*$'+c+',f'
        mk[c+',line_1_truncexp'] = '$env,line_1_truncexp'
        mk[c+',line_2_truncexp'] = '$env,line_1_truncexp'
        mk[c+',line_3_truncexp'] = '$env,line_1_truncexp'
        mk[c+',line_1_wl0'] = '$env,line_1_wl0'
        mk[c+',line_2_wl0'] = '$env,line_2_wl0'
        mk[c+',line_3_wl0'] = '$env,line_3_wl0'
    doNotFit.extend(['env,f', 'ABS1', 'ABS2', 'ABS3'])
oi.doFit(mk, doNotFit=doNotFit)
#pmoired.oimodels.computeLambdaParams(mk)
oi.show(imFov=6, imWl0=[2.20, 2.312], imPow=0.5)

In [ ]:
plt.figure(oi.fig-1)
plt.savefig('RW_Cep_Kband_model.pdf')
plt.figure(oi.fig-2)
plt.savefig('RW_Cep_Kband_fit.pdf')